In [2]:
import os
import openai

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
openai.api_key = OPENAI_API_KEY

In [8]:
messages = [{"role": "system", "content": "Hello World"}]
response = openai.ChatCompletion.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    n=1,
    stop=None,
)
response.choices[0].message.content

'Hello! How can I assist you today?'

In [15]:
import json

def create_file(path, content):
    with open(path, "w") as f:
        f.write(content)

tools = [
    {
        "type": "function",
        "function": {
            "name": "create_file",
            "description": "Create a file with the given name and content",
            "parameters": {
                "type": "object",
                "properties": {
                    "path": {
                        "type": "string",
                        "description": "The relative path to the file to create",
                    },
                    "content": {
                        "type": "string",
                        "description": "The content of the file to create",
                    },
                },
                "required": ["path", "content"],
            },
        },
    }
]

messages = [{"role": "system", "content": """
             Create me a file called 'hello.txt' with the content 'Hello World'
             """}]
response = openai.ChatCompletion.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    n=1,
    stop=None,
    tools=tools,
)
message = response.choices[0].message

if message.tool_calls and message.tool_calls[0].function.name == "create_file":
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    create_file(arguments["path"], arguments["content"])
else:
    print(message)